In [12]:
import cv2 # Importing cv2 library
import numpy as np # Importing numpy library
from time import sleep
import csv

with open("rubikThreshold.csv", 'r') as infile:
    '''
    number	label
    0	green
    1	white
    2	yellow
    3	red
    4	orange
    5	blue
    '''
    thresholds = list(csv.reader(infile))  # 6x6 array, each row contains 1 color, hsv min & max
    thresholds = np.asarray(thresholds, dtype=int)


cap = cv2.VideoCapture(1) #Choose to capture laptopcam or external webcam
out = cv2.VideoWriter('filename.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, (640, 480)) 

while True: #continues forever
    retrieveval, frame_org = cap.read()
    h, w, ch = frame_org.shape
    d = 360
    frame = cv2.bitwise_or(np.zeros(frame_org.shape, np.uint8), frame_org)
    frame = frame[(h-d)//2:(h+d)//2, (w-d)//2:(w+d)//2]
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) #convert BGR to HSV

    '''RETR_EXTERNAL --> retrieves only the extreme outer contours, concentric circles
    only outermost circle is taken

    RETR_LIST --> gives all the contours, even if found inside one another
    
    RETR_CCOMP --> gives all the contours, outer and inner
    
    RETR_TREE--> retrieves all of the contours and reconstructs a full rlnshp 
    of how all the contours are related to each other
       
    '''

    '''
    CV_CHAIN_APPROX_NONE stores absolutely all the contour points. 
    That is, any 2 subsequent points (x1,y1) and (x2,y2) of the contour will be 
    either horizontal, vertical or diagonal neighbors, that is, max(abs(x1-x2),abs(y2-y1))==1.

    CV_CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments
    and leaves only their end points. For example, an up-right rectangular 
    contour is encoded with 4 points.

    CV_CHAIN_APPROX_TC89_L1,CV_CHAIN_APPROX_TC89_KCOS applies one of the 
    flavors of the Teh-Chin chain approximation algorithm. 
    
    ''' 
    notable = []
    for i in range(6):
        color = thresholds[i]
        col_type = color[0]
        low = color[1:4]
        high = color[4:]
        height, width, channels = frame.shape
        if col_type == -2:
            mask = cv2.inRange(frame, low, high)
        else:
            mask = cv2.inRange(hsv_frame, low, high)
        contours, retrieveval = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for con in contours:
            if cv2.contourArea(con)>5000:
                (x,y,w,h) = cv2.boundingRect(con)
                
                font                   = cv2.FONT_HERSHEY_SIMPLEX
                bottomLeftCornerOfText = (x+10, y+30)
                fontScale              = 1
                fontColor              = (0,0,0)
                lineType               = 2
                cv2.putText(frame, str(i), 
                    bottomLeftCornerOfText, 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
                notable.append((x//(width//4), y//(height//4), i))
        

    #Showing the frame and masks
    cv2.imshow("Tracking1", frame)
    cv2.imshow("Tracking2", frame_org)
    out.write(frame_org)
    
    key = cv2.waitKey(1)
    if key & 0xFF ==ord('q'):
        cap.release() #release resources
        cv2.destroyAllWindows() #get rid of all live windows
        out.release()
        
        break
    elif key & 0xFF == ord('k'):
        for i in range(6):
            print(i)
            color = thresholds[i]
            col_type = color[0]
            low = color[1:4]
            high = color[4:]
            print(low, high)
            if col_type == -2:
                mask = cv2.inRange(frame, low, high)
            else:
                mask = cv2.inRange(hsv_frame, low, high)
            contours, retrieveval = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            '''if i==3 or i==4:
                
                print(i)
                contours = sorted(contours, key = lambda x:cv2.contourArea(x), reverse=True)
                for con in contours:
                    print(cv2.contourArea(con))'''
            for con in contours:
                if cv2.contourArea(con)>4500:
                    (x,y,w,h) = cv2.boundingRect(con)
                    print(x,y)
    elif key & 0xFF == ord('b'):
        path = "./images1/"
        if len(notable)!=9:
            continue
        answers = sorted(notable,key=lambda x: (x[1],x[0]))
        labels = ','.join([str(x[2]) for x in answers])
        maxnum = 0
        for item in os.listdir(path):
            if labels in item:
                maxnum = max(maxnum, int(item[18:-4]))
        cv2.imwrite(path+labels+'_0.png', frame_org)

        
        

In [ ]:
import cv2 # Importing cv2 library
import numpy as np # Importing numpy library
from time import sleep
import csv

cap = cv2.VideoCapture(1) #Choose to capture laptopcam or external webcam

retrieveval, frame_org = cap.read()
h, w = frame_org.shape
print(len(frame_org))

cv2.imshow("image", frame_org)
if cv2.waitKey(1) & 0xFF ==ord('q'):
    cap.release() #release resources
    cv2.destroyAllWindows() #get rid of all live windows